## Gather Data 

In [200]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import matplotlib.pyplot as plt
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 

**First, we would scrape the top 100 movies of all time from Rotten Tomatoes**

In [305]:
URL = "https://www.rottentomatoes.com/top/bestofrt/" #URL where the table for the top 100 movies exist
page = requests.get(URL) 
soup = bs(page.content, 'lxml') #Creating the soup
movie_elems = soup.find_all(class_='table') #Finding the table in the soup
# headers=[]
# for header in movie_elems[0].findAll('th'): #Getting the headers of the table
#     headers.append(header.text)


In [306]:
table = movie_elems[0].findAll('tr') #Creating this variable to find the row observations for each movie

In [307]:
ranks = []
for rows in table[1:]: #Loop through the 'table' to get the rank
    for rank in rows.find('td', class_ = 'bold'):
        ranks.append(rank)

In [308]:
movies = []
for rows in table[1:]: #Loop through the 'table' to get the movie names
    for movie in rows.find('a'):
        movies.append(movie)

In [309]:
ratings = []
for rows in table[1:]: #Loop through the 'table' to get the critic rating
    for rating in rows.find('span', class_='tMeterScore'):
        ratings.append(rating)

In [310]:
number_reviews = []
for rows in table[1:]: #Loop through the 'table' to get the number of critic reviews
    for review in rows.find('td', class_='right hidden-xs'):
        number_reviews.append(review)

In [311]:
movie_reviews = {'Rank': ranks,  'Movie_Title': movies, 'Critic_Rating': ratings, 'Number_of_Critic_Reviews': number_reviews}

In [312]:
df_t = pd.DataFrame(movie_reviews)

**We would now gather all the data like synopsis, critic rating, number of critic reviews, Audience rating and number of Audience rating for each of the movie in the top 100 list**

For this exercise, we would be using Selenium. Selenium is a powerful tool that helps us in automating the web browser. In combination with beautiful soup it becomes one of the most powerful web scraping tools

Please download the chromedriver from https://chromedriver.chromium.org/ and then set the proper executable path.

In [228]:
#Initiating a webdriver of Chrome
driver = webdriver.Chrome(executable_path='/Users/akshaygupta/Downloads/chromedriver')
driver.get("https://www.rottentomatoes.com/top/bestofrt/")
driver.implicitly_wait(10)
df = pd.DataFrame() #Creating an empty dataframe to store information
for i in range(1,101):
    driver.find_element_by_xpath('//*[@id="top_movies_main"]/div/table/tbody/tr['+str(i)+']/td[3]/a').click()
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME,'mop-ratings-wrap__percentage')))
    html = driver.page_source
    soup = bs(html, 'lxml')#Creating the soup
    section = soup.find_all(class_='mop-ratings-wrap__info') #Setting the section variable to be a subpart of the page for ease in reading the code
    synopsis = section[0].find('p').text #Scraping the synopsis
    score_section = section[0].find_all('div') #Subset of section for ease in reading the code
    critic_rat = score_section[0].find_all('span')[1].text #Scraping the critic rating
    critic_num = score_section[1].find_all('small')[0].text #Scraping the number of critic reviews
    audience_rat = score_section[2].find_all('span')[1].text #Scraping the audience rating
    audience_num = score_section[3].find('strong').text #Scraping the number of audience reviews
    temp = pd.DataFrame({
        'Synopsis': synopsis.strip(),
        'Critic_Rating': critic_rat.strip(),
        'Number_of_Critic_Reviews': critic_num.strip(),
        'Audience_Rating': audience_rat.strip(),
        'Number_of_Audience_Reviews': audience_num.strip()[audience_num.find(':')+2:]
    }, index=[i]) #Making a temporary pandas DataFrame to store information for each movie which resets after each loop
    df = pd.concat([df, temp]) #Joining the temp to the empty DataFrame created outside this loop
    driver.back()
driver.quit()

## Assess 

In this section, we would assess the data we have scraped from Rotten Tomatoes and note down any data inconsistencies we see

**Assessing df_t**

In [209]:
df_t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Rank                      100 non-null    object
 1   Movie_Title               100 non-null    object
 2   Critic_Rating             100 non-null    object
 3   Number_of_Critic_Reviews  100 non-null    object
dtypes: object(4)
memory usage: 3.2+ KB


In [210]:
df_t.head()

,Rank,Movie_Title,Critic_Rating,Number_of_Critic_Reviews
0,1.,\n Black Panther (2018),96%,512
1,2.,\n Avengers: Endgame (2019),94%,528
2,3.,\n Us (2019),93%,533
3,4.,\n Toy Story 4 (2019),97%,443
4,5.,\n Lady Bird (2017),99%,391


In [211]:
df_t.tail()

,Rank,Movie_Title,Critic_Rating,Number_of_Critic_Reviews
95,96.,\n The Lady Eve (1941),100%,47
96,97.,"\n The Godfather, Part II (1974)",98%,83
97,98.,\n Chinatown (1974),99%,75
98,99.,\n Won't You Be My Neighbor? (2018),97%,249
99,100.,\n The Babadook (2014),98%,238


- Data Types for all the column is object
- \n exists in all the rows of the Title column
- % sign exists at the end of the rating
- . exists at the end of the ranking
- Year of the movie is within the title

**Assessing df**

In [176]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 1 to 100
Data columns (total 5 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Synopsis                    100 non-null    object
 1   Critic Rating               100 non-null    object
 2   Number of Critic Reviews    100 non-null    object
 3   Audience Rating             100 non-null    object
 4   Number of Audience Reviews  100 non-null    object
dtypes: object(5)
memory usage: 4.7+ KB


In [177]:
df.head()

,Synopsis,Critic Rating,Number of Critic Reviews,Audience Rating,Number of Audience Reviews
Rank,,,,,
1,Black Panther elevates superhero cinema to thr...,96%,512,79%,"88,211"
2,"Exciting, entertaining, and emotionally impact...",94%,528,90%,"70,313"
3,"With Jordan Peele's second inventive, ambitiou...",93%,533,59%,"13,108"
4,"Heartwarming, funny, and beautifully animated,...",97%,443,94%,"53,146"
5,Lady Bird delivers fresh insights about the tu...,99%,391,79%,"22,595"


In [178]:
df.tail()

,Synopsis,Critic Rating,Number of Critic Reviews,Audience Rating,Number of Audience Reviews
Rank,,,,,
96,"A career highlight for Preston Sturges, The La...",100%,47,87%,"7,550"
97,Drawing on strong performances by Al Pacino an...,98%,83,97%,"411,450"
98,As bruised and cynical as the decade that prod...,99%,75,93%,"77,907"
99,Won't You Be My Neighbor? takes a fittingly pa...,97%,249,94%,"4,382"
100,The Babadook relies on real horror rather than...,98%,238,72%,"39,109"


- All the columns are of object type
- % sign in column Critic Rating and Audience Rating
- , in Number of Audience Reviews

## Cleaning 

#### Top 100 movies table (df_t)

**Cleaning df_t**

- Remove % sign at the end of the critic rating
- Remove . from ranking
- Convert Rank, Critic Rating and No. of Critic Reviews to int

In [332]:
df_t_copy = df_t.copy()

In [333]:
df_t_copy['Critic_Rating'] = df_t_copy['Critic_Rating'].apply(lambda x: x[:-1])

In [334]:
df_t_copy['Rank'] = df_t_copy['Rank'].apply(lambda x: x[:-1])

In [335]:
df_t_copy['Critic_Rating'] = df_t_copy['Critic_Rating'].apply(lambda x: int(x))
df_t_copy['Rank'] = df_t_copy['Rank'].apply(lambda x: int(x))
df_t_copy['Number_of_Critic_Reviews'] = df_t_copy['Number_of_Critic_Reviews'].apply(lambda x: int(x))

- Remove \n from movie title
- Convert movie title to string

In [336]:
df_t_copy['Movie_Title'] = df_t_copy['Movie_Title'].apply(lambda x: str(x.strip()))

- Separate the year of the movie from the Movie Title
- Convert Year of Release to int type

In [337]:
df_t_copy['Year_of_Release'] = df_t_copy['Movie_Title'].apply(lambda x: int(x[-5:-1]))
df_t_copy['Movie_Title'] = df_t_copy['Movie_Title'].apply(lambda x: x[:-6])

Converting Year of Release to int type makes sense here since it is only the year. Had the release date been with day and month, we would have converted it to datetime object for ease of analysis.

**Testing df_t**

In [338]:
df_t_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Rank                      100 non-null    int64 
 1   Movie_Title               100 non-null    object
 2   Critic_Rating             100 non-null    int64 
 3   Number_of_Critic_Reviews  100 non-null    int64 
 4   Year_of_Release           100 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 4.0+ KB


In [339]:
df_t_copy.head()

,Rank,Movie_Title,Critic_Rating,Number_of_Critic_Reviews,Year_of_Release
0,1,Black Panther,96,512,2018
1,2,Avengers: Endgame,94,528,2019
2,3,Us,93,533,2019
3,4,Toy Story 4,97,443,2019
4,5,Lady Bird,99,391,2017


In [340]:
df_t_copy.tail()

,Rank,Movie_Title,Critic_Rating,Number_of_Critic_Reviews,Year_of_Release
95,96,The Lady Eve,100,47,1941
96,97,"The Godfather, Part II",98,83,1974
97,98,Chinatown,99,75,1974
98,99,Won't You Be My Neighbor?,97,249,2018
99,100,The Babadook,98,238,2014


In [342]:
print(df_t_copy['Rank'].mean(), df_t_copy['Critic_Rating'].mean(), df_t_copy['Number_of_Critic_Reviews'].mean(), df_t_copy['Year_of_Release'].mean())

50.5 96.61 256.71 1988.68


#### Top 100 movies (df)

**Cleaning df**

Since we are going to join this table with the df_t table, we do not need the Critic Rating and the Number of Critic Reviews as they are already recorded in df_t

Creating a copy of df so that we don't mess with the original data

In [343]:
df_copy = df.copy()

In [344]:
df_copy = df.drop(['Critic_Rating', 'Number_of_Critic_Reviews'], axis=1)

In [345]:
df_copy.reset_index(inplace=True)

- Removing all the % sign from the Audience Rating column
- Remove , from the Number of Audience Reviews column
- Convert Rank, Audience Rating and Number of Audience Reviews to int type

In [346]:
df_copy

,index,Synopsis,Audience_Rating,Number_of_Audience_Reviews
0,1,Black Panther elevates superhero cinema to thr...,79%,"88,211"
1,2,"Exciting, entertaining, and emotionally impact...",90%,"70,313"
2,3,"With Jordan Peele's second inventive, ambitiou...",59%,"13,108"
3,4,"Heartwarming, funny, and beautifully animated,...",94%,"53,146"
4,5,Lady Bird delivers fresh insights about the tu...,79%,"22,595"
...,...,...,...,...
95,96,"A career highlight for Preston Sturges, The La...",87%,"7,550"
96,97,Drawing on strong performances by Al Pacino an...,97%,"411,450"
97,98,As bruised and cynical as the decade that prod...,93%,"77,907"
98,99,Won't You Be My Neighbor? takes a fittingly pa...,94%,"4,382"


In [347]:
df_copy['Audience_Rating'] = df_copy['Audience_Rating'].apply(lambda x: int(x[:-1]))
df_copy['Number_of_Audience_Reviews'] = df_copy['Number_of_Audience_Reviews'].apply(lambda x: int(x.replace(',','')))

- Convert index to Rank
- Convert Rank to int type

In [348]:
df_copy.rename({'index':'Rank'}, axis=1, inplace=True)

In [349]:
df_copy['Rank'] = df_copy['Rank'].apply(lambda x: int(x))

- Convert Synopsis to str

In [350]:
df_copy['Synopsis'] = df_copy['Synopsis'].apply(lambda x: str(x))

**Testing df_copy**

In [351]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Rank                        100 non-null    int64 
 1   Synopsis                    100 non-null    object
 2   Audience_Rating             100 non-null    int64 
 3   Number_of_Audience_Reviews  100 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 3.2+ KB


In [352]:
df_copy.head()

,Rank,Synopsis,Audience_Rating,Number_of_Audience_Reviews
0,1,Black Panther elevates superhero cinema to thr...,79,88211
1,2,"Exciting, entertaining, and emotionally impact...",90,70313
2,3,"With Jordan Peele's second inventive, ambitiou...",59,13108
3,4,"Heartwarming, funny, and beautifully animated,...",94,53146
4,5,Lady Bird delivers fresh insights about the tu...,79,22595


In [353]:
df_copy.tail()

,Rank,Synopsis,Audience_Rating,Number_of_Audience_Reviews
95,96,"A career highlight for Preston Sturges, The La...",87,7550
96,97,Drawing on strong performances by Al Pacino an...,97,411450
97,98,As bruised and cynical as the decade that prod...,93,77907
98,99,Won't You Be My Neighbor? takes a fittingly pa...,94,4382
99,100,The Babadook relies on real horror rather than...,72,39109


#### Merging the table 

In [355]:
df_master = df_t_copy.merge(df_copy, how='inner', on='Rank')

Rearranging the columns for ease in readability. This step is not required but that's just the OCD me!

In [356]:
df_master= df_master[['Rank', 'Movie_Title', 'Year_of_Release', 'Synopsis', 'Critic_Rating', 'Number_of_Critic_Reviews', 'Audience_Rating', 'Number_of_Audience_Reviews']]

In [357]:
df_master

,Rank,Movie_Title,Year_of_Release,Synopsis,Critic_Rating,Number_of_Critic_Reviews,Audience_Rating,Number_of_Audience_Reviews
0,1,Black Panther,2018,Black Panther elevates superhero cinema to thr...,96,512,79,88211
1,2,Avengers: Endgame,2019,"Exciting, entertaining, and emotionally impact...",94,528,90,70313
2,3,Us,2019,"With Jordan Peele's second inventive, ambitiou...",93,533,59,13108
3,4,Toy Story 4,2019,"Heartwarming, funny, and beautifully animated,...",97,443,94,53146
4,5,Lady Bird,2017,Lady Bird delivers fresh insights about the tu...,99,391,79,22595
...,...,...,...,...,...,...,...,...
95,96,The Lady Eve,1941,"A career highlight for Preston Sturges, The La...",100,47,87,7550
96,97,"The Godfather, Part II",1974,Drawing on strong performances by Al Pacino an...,98,83,97,411450
97,98,Chinatown,1974,As bruised and cynical as the decade that prod...,99,75,93,77907
98,99,Won't You Be My Neighbor?,2018,Won't You Be My Neighbor? takes a fittingly pa...,97,249,94,4382


In [358]:
df_master.to_csv('Top 100 Movie Reviews.csv')